In [2]:
from joblib import dump
from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
data = read_csv("iris.csv")
data.head()

,panjang kelopak,lebar kelopak,panjang mahkota,lebar mahkota,spesies
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    data.drop(['spesies'],axis=1), data['spesies'], test_size=0.2, train_size=0.8
)

In [5]:
knn = KNeighborsClassifier()

knn = knn.fit(x_train, y_train)
hasil = knn.predict(x_test)

print(confusion_matrix(hasil, y_test))
print(classification_report(hasil, y_test))

[[13  0  0]
 [ 0  6  0]
 [ 0  1 10]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        13
Iris-versicolor       0.86      1.00      0.92         6
 Iris-virginica       1.00      0.91      0.95        11

       accuracy                           0.97        30
      macro avg       0.95      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30



In [6]:
dump(knn, "models/iris.pkl")

['models/iris.pkl']

# Image

In [8]:
# Third party libraries
import matplotlib.pyplot as plt
import torch

from torchvision import models, transforms
from sklearn.metrics import (
    classification_report,
    confusion_matrix)
    
# Local module
from helper import (
    image_processing,
    cnn_model,
    model_training,
    model_testing,
    adversarial_training)

In [12]:
device = torch.device("mps")
torch.manual_seed(123)

In [38]:
train_data, test_data = image_processing(
    train_path = "data/latih",
    test_path = "data/uji",
    batch_train = 100,
    batch_test = 10)

In [39]:
int_to_label = lambda label_in_tensor : [
    str(label).replace('0', 'cat').replace('1', 'dog')
    for label in label_in_tensor.tolist()]

In [44]:
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [46]:
os.environ['TORCH_HOME'] = 'venv/Lib/site-packages/torch'
shufflenetv2, loss_fn, optimizer = cnn_model(
    models.shufflenet_v2_x1_0(pretrained=True))

Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to venv/Lib/site-packages/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth
100.0%


In [47]:
shufflenetv2 = model_training(
    batch_size = 100,
    data_train = train_data,
    num_epochs = 10,
    model = shufflenetv2,
    loss_func = loss_fn,
    optimizer = optimizer)

Epoch: 1, Loss: 0.5287
Epoch: 2, Loss: 0.3823
Epoch: 3, Loss: 0.3277
Epoch: 4, Loss: 0.2928
Epoch: 5, Loss: 0.2729
Epoch: 6, Loss: 0.2441
Epoch: 7, Loss: 0.2279
Epoch: 8, Loss: 0.2278
Epoch: 9, Loss: 0.2016
Epoch: 10, Loss: 0.1824


In [48]:
conf_matrx, class_report = model_testing(test_data, shufflenetv2)

print(f'Confusion Matrix:\n{conf_matrx}\n')
print(f'Classification report:\n{class_report}')

Confusion Matrix:
[[465  52]
 [ 97 398]]

Classification report:
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       517
           1       0.88      0.80      0.84       495

    accuracy                           0.85      1012
   macro avg       0.86      0.85      0.85      1012
weighted avg       0.86      0.85      0.85      1012



In [49]:
dump(shufflenetv2, "models/image.pkl")

['models/image.pkl']

In [52]:
for image, labels in test_data:
    break

In [55]:
image[0].shape

torch.Size([3, 100, 100])

In [59]:
test = image[0].view(-1, 3, 100, 100)
outputs = shufflenetv2(test)
predicted = torch.max(outputs, 1)[1][0]

In [60]:
predicted

tensor(1)

In [66]:
import numpy as np

In [108]:
from skimage import io
from skimage import transform

In [116]:
img = io.imread('dog_6502.jpg')
resized_img = transform.resize(img, (100, 100))
outputs = shufflenetv2(torch.tensor([np.transpose(resized_img).astype(np.float32)]))
predicted = torch.max(outputs, 1)[1][0]
predicted

tensor(1)

In [107]:
outputs

tensor([[-0.0385, -3.2750]], grad_fn=<LogSoftmaxBackward0>)

In [115]:
outputs

tensor([[-1.1709, -0.3712]], grad_fn=<LogSoftmaxBackward0>)